In [14]:
import pandas as pd
import numpy as np
import os

# Hàm split_dataframe giữ nguyên, không cần thay đổi.
def split_dataframe(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Nhận một DataFrame, tách nó thành hai phần (đúng và cần crawl lại)
    dựa trên các điều kiện đã xác định và trả về hai DataFrame đã được
    loại bỏ trùng lặp.
    """
    df_copy = df.copy()

    # --- CẢI TIẾN: Chuẩn hóa dữ liệu trước khi so sánh ---
    main_link_correct = df_copy['MainLinkCorrect'].fillna('').astype(str).str.strip().str.upper()
    date_correct = df_copy['ConferenceDateCorrect'].fillna('').astype(str).str.strip().str.upper()
    location_correct = df_copy['LocationCorrect'].fillna('').astype(str).str.strip().str.upper()

    # Điều kiện 1: Sử dụng các cột đã được chuẩn hóa
    condition1 = (
        (main_link_correct == 'FALSE') |
        (date_correct == 'FALSE') |
        (location_correct == 'FALSE')
    )

    # Điều kiện 2: Một trong các trường Percent khác 100 VÀ KHÔNG TRỐNG
    percent_cols = ['Percent', 'Percent_1', 'Percent_2', 'Percent_3', 'Percent_4']
    
    for col in percent_cols:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

    condition2_parts = [df_copy[col].notna() & (df_copy[col] != 100) for col in percent_cols]
    
    condition2 = False
    for part in condition2_parts:
        condition2 = condition2 | part

    # Kết hợp hai điều kiện để tìm ra tất cả các dòng cần crawl lại
    recrawl_condition = condition1 | condition2

    # Tách DataFrame
    recrawl_df = df_copy[recrawl_condition]
    correct_df = df_copy[~recrawl_condition]

    # Loại bỏ trùng lặp cho mỗi phần
    subset_cols = ['title', 'acronym']
    unique_recrawl_df = recrawl_df.drop_duplicates(subset=subset_cols, keep='first')
    unique_correct_df = correct_df.drop_duplicates(subset=subset_cols, keep='first')

    return unique_correct_df, unique_recrawl_df


# --- HÀM NÀY ĐƯỢC CẬP NHẬT ĐỂ NHẬN THÊM THAM SỐ ---
def process_file_list(
    input_files: list, 
    output_dir: str,
    aggregated_correct_path: str, 
    aggregated_recrawl_path: str,
    recrawled_files: list = None # Thêm tham số cho các file đã recrawl
):
    """
    Xử lý một danh sách các file CSV, đối chiếu với kết quả đã recrawl,
    và tạo ra hai file tổng hợp cuối cùng.
    """
    all_correct_dfs = []
    all_recrawl_dfs = []

    os.makedirs(output_dir, exist_ok=True)

    # --- BƯỚC 1: XỬ LÝ CÁC FILE INPUT BAN ĐẦU ---
    for file_path in input_files:
        print(f"\n--- Đang xử lý file input: {file_path} ---")
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str)
            print(f"Đã đọc thành công {len(df)} dòng.")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file '{file_path}'. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Đã xảy ra lỗi khi đọc file '{file_path}': {e}. Bỏ qua file này.")
            continue

        correct_df, recrawl_df = split_dataframe(df)
        all_correct_dfs.append(correct_df)
        all_recrawl_dfs.append(recrawl_df)

    # --- BƯỚC 2: GỘP VÀ XỬ LÝ BỔ SUNG BAN ĐẦU ---
    if not all_correct_dfs and not all_recrawl_dfs:
        print("\nKhông có dữ liệu từ các file input để xử lý. Kết thúc.")
        return

    print("\n--- Đang tổng hợp kết quả từ các file input ---")
    master_correct_df = pd.concat(all_correct_dfs, ignore_index=True) if all_correct_dfs else pd.DataFrame()
    master_recrawl_df = pd.concat(all_recrawl_dfs, ignore_index=True) if all_recrawl_dfs else pd.DataFrame()

    # Di chuyển các dòng có ghi chú (Note)
    if 'Note' in master_correct_df.columns:
        note_exists_condition = master_correct_df['Note'].notna()
        rows_to_move = master_correct_df[note_exists_condition]
        if not rows_to_move.empty:
            print(f"Tìm thấy {len(rows_to_move)} dòng có ghi chú. Di chuyển sang danh sách recrawl.")
            master_recrawl_df = pd.concat([master_recrawl_df, rows_to_move], ignore_index=True)
            master_correct_df = master_correct_df[~note_exists_condition]

    # --- BƯỚC 3 (MỚI): ĐỐI CHIẾU VỚI CÁC FILE ĐÃ RECRAWL ---
    if recrawled_files:
        print("\n--- Đang đối chiếu với kết quả đã recrawl ---")
        recrawled_dfs_list = []
        for r_file in recrawled_files:
            try:
                df = pd.read_csv(r_file, encoding='utf-8-sig', na_values=['', ' '], dtype=str)
                recrawled_dfs_list.append(df)
                print(f"Đã đọc file đã recrawl: {r_file}")
            except FileNotFoundError:
                print(f"Cảnh báo: Không tìm thấy file đã recrawl '{r_file}'. Bỏ qua.")
        
        if recrawled_dfs_list:
            # Gộp tất cả các file đã recrawl thành một DataFrame lớn
            recrawled_master_df = pd.concat(recrawled_dfs_list, ignore_index=True)
            
            # Dùng lại hàm split_dataframe để xác định những dòng nào giờ đã ĐÚNG
            newly_correct_df, _ = split_dataframe(recrawled_master_df)
            
            if not newly_correct_df.empty:
                print(f"Tìm thấy {len(newly_correct_df)} dòng đã được sửa đúng sau khi recrawl.")
                
                # Tạo một set các khóa (title, acronym) của các dòng đã được sửa đúng để tra cứu nhanh
                corrected_keys = set(zip(
                    newly_correct_df['title'].str.strip(), 
                    newly_correct_df['acronym'].str.strip()
                ))
                
                # Loại bỏ các dòng đã được sửa đúng khỏi danh sách recrawl tổng
                # Tạo một mask boolean để xác định dòng nào cần loại bỏ
                recrawl_keys = zip(
                    master_recrawl_df['title'].str.strip(),
                    master_recrawl_df['acronym'].str.strip()
                )
                is_now_correct_mask = [key in corrected_keys for key in recrawl_keys]
                
                # Giữ lại những dòng KHÔNG có trong danh sách đã sửa đúng
                master_recrawl_df = master_recrawl_df[~pd.Series(is_now_correct_mask).values]
                
                # Thêm các dòng vừa được sửa đúng vào danh sách correct tổng
                master_correct_df = pd.concat([master_correct_df, newly_correct_df], ignore_index=True)
            else:
                print("Không có dòng nào được sửa đúng trong các file đã recrawl.")

    # --- BƯỚC 4: LOẠI BỎ TRÙNG LẶP LẦN CUỐI VÀ LƯU FILE ---
    print("\n--- Hoàn tất và lưu file tổng hợp cuối cùng ---")
    
    # Xử lý và lưu file CORRECT tổng
    if not master_correct_df.empty:
        final_correct_df = master_correct_df.drop_duplicates(subset=['title', 'acronym'], keep='first')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng đúng vào '{aggregated_correct_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'đúng' nào. Đã tạo file rỗng '{aggregated_correct_path}'")

    # Xử lý và lưu file RECRAWL tổng
    if not master_recrawl_df.empty:
        final_recrawl_df = master_recrawl_df.drop_duplicates(subset=['title', 'acronym'], keep='first')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng cần crawl lại vào '{aggregated_recrawl_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'cần recrawl' nào. Đã tạo file rỗng '{aggregated_recrawl_path}'")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # 1. Danh sách các file input ban đầu
    input_csv_files = [
        './src/conference/evaluate/batch2.csv',
        './src/conference/evaluate/batch3.csv',
        './src/conference/evaluate/batch8.csv',
        './src/conference/evaluate/batch12.csv',
        './src/conference/evaluate/batch13.csv',
        './src/conference/evaluate/batch16.csv',
        './src/conference/evaluate/batch19.csv',
    ]

    # 2. (MỚI) Danh sách các file là kết quả của lần recrawl trước đó
    # Để trống danh sách này nếu đây là lần chạy đầu tiên: recrawled_results_files = []
    recrawled_results_files = [
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        # Thêm các file kết quả recrawl khác vào đây
    ]

    # 3. Thư mục để chứa các file output riêng lẻ (không thay đổi)
    individual_output_directory = './src/conference/evaluate/individual_outputs'

    # 4. Đường dẫn cho 2 file tổng hợp cuối cùng (không thay đổi)
    aggregated_correct_file = './src/conference/evaluate/ALL_BATCHES_correct_final.csv'
    aggregated_recrawl_file = './src/conference/evaluate/ALL_BATCHES_recrawl_final.csv'

    # 5. Gọi hàm xử lý chính với tham số mới
    process_file_list(
        input_files=input_csv_files,
        output_dir=individual_output_directory,
        aggregated_correct_path=aggregated_correct_file,
        aggregated_recrawl_path=aggregated_recrawl_file,
        recrawled_files=recrawled_results_files # Truyền danh sách file đã recrawl
    )


--- Đang xử lý file input: ./src/conference/evaluate/batch2.csv ---
Đã đọc thành công 47 dòng.

--- Đang xử lý file input: ./src/conference/evaluate/batch3.csv ---
Đã đọc thành công 46 dòng.

--- Đang xử lý file input: ./src/conference/evaluate/batch8.csv ---
Đã đọc thành công 48 dòng.

--- Đang xử lý file input: ./src/conference/evaluate/batch12.csv ---
Đã đọc thành công 38 dòng.

--- Đang xử lý file input: ./src/conference/evaluate/batch13.csv ---
Đã đọc thành công 46 dòng.

--- Đang xử lý file input: ./src/conference/evaluate/batch16.csv ---
Đã đọc thành công 46 dòng.

--- Đang xử lý file input: ./src/conference/evaluate/batch19.csv ---
Đã đọc thành công 46 dòng.

--- Đang tổng hợp kết quả từ các file input ---
Tìm thấy 21 dòng có ghi chú. Di chuyển sang danh sách recrawl.

--- Đang đối chiếu với kết quả đã recrawl ---
Đã đọc file đã recrawl: ./src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv
Đã đọc file đã recrawl: ./src/conference/evaluate/evaluate_recrawl_batch_8_lan

In [ ]:
import pandas as pd
import os

def split_df_by_note(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Tách một DataFrame thành hai phần dựa trên sự tồn tại của giá trị trong cột 'Note'.

    - Dữ liệu "cần recrawl": Các dòng có giá trị trong cột 'Note'.
    - Dữ liệu "đúng": Các dòng không có giá trị trong cột 'Note' (trống hoặc NaN).

    Args:
        df (pd.DataFrame): DataFrame đầu vào.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: Một tuple chứa (correct_df, recrawl_df).
    """
    # Kiểm tra an toàn để đảm bảo cột 'Note' tồn tại
    if 'note' not in df.columns:
        print("Cảnh báo: Không tìm thấy cột 'Note'. Coi như tất cả các dòng đều 'đúng'.")
        # Trả về toàn bộ DataFrame là 'correct' và một DataFrame rỗng là 'recrawl'
        return df.copy(), pd.DataFrame(columns=df.columns)

    # Điều kiện: cột 'Note' có giá trị (không phải NaN và không phải chuỗi rỗng)
    # .notna() là cách kiểm tra tốt nhất cho NaN/None.
    # Thêm điều kiện .str.strip() != '' để loại bỏ các ô chỉ chứa khoảng trắng.
    condition_recrawl = df['Note'].notna() & (df['Note'].astype(str).str.strip() != '')

    # Tách DataFrame dựa trên điều kiện
    recrawl_df = df[condition_recrawl].copy()
    correct_df = df[~condition_recrawl].copy()

    return correct_df, recrawl_df

def process_files_by_note(
    input_files: list, 
    output_dir: str,
    aggregated_correct_path: str, 
    aggregated_recrawl_path: str
):
    """
    Xử lý một danh sách file, lọc từng file dựa trên cột 'Note',
    lưu output riêng lẻ và hai file tổng hợp cuối cùng.

    Args:
        input_files (list): Danh sách đường dẫn đến các file CSV đầu vào.
        output_dir (str): Thư mục để lưu các file output riêng lẻ.
        aggregated_correct_path (str): Đường dẫn file tổng hợp dữ liệu đúng.
        aggregated_recrawl_path (str): Đường dẫn file tổng hợp dữ liệu cần crawl lại.
    """
    all_correct_dfs = []
    all_recrawl_dfs = []

    os.makedirs(output_dir, exist_ok=True)

    for file_path in input_files:
        print(f"\n--- Đang xử lý file: {file_path} ---")
        try:
            # Đọc file với dtype=str để đảm bảo cột 'Note' được đọc đúng dạng chuỗi
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
            print(f"Đã đọc thành công {len(df)} dòng.")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file '{file_path}'. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Đã xảy ra lỗi khi đọc file '{file_path}': {e}. Bỏ qua file này.")
            continue

        # Gọi hàm logic cốt lõi để tách dữ liệu
        correct_df, recrawl_df = split_df_by_note(df)
        
        print(f"Kết quả tách: {len(correct_df)} dòng không có ghi chú (đúng), {len(recrawl_df)} dòng có ghi chú (cần recrawl).")

        # Thêm các DataFrame đã xử lý vào danh sách tổng hợp
        if not correct_df.empty:
            all_correct_dfs.append(correct_df)
        if not recrawl_df.empty:
            all_recrawl_dfs.append(recrawl_df)

        # --- Lưu các file output riêng lẻ ---
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        individual_correct_path = os.path.join(output_dir, f"{base_name}_note_correct.csv")
        individual_recrawl_path = os.path.join(output_dir, f"{base_name}_note_recrawl.csv")

        correct_df.to_csv(individual_correct_path, index=False, encoding='utf-8-sig')
        recrawl_df.to_csv(individual_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu output riêng lẻ vào thư mục '{output_dir}'")

    # --- Gộp và lưu các file tổng hợp ---
    print("\n--- Đang tổng hợp tất cả kết quả ---")
    
    # Gộp, loại bỏ trùng lặp và lưu file CORRECT tổng
    if all_correct_dfs:
        master_correct_df = pd.concat(all_correct_dfs, ignore_index=True)
        final_correct_df = master_correct_df.drop_duplicates(subset=['title', 'acronym'], keep='first')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng đúng (không có ghi chú) vào '{aggregated_correct_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'đúng' nào. Đã tạo file rỗng '{aggregated_correct_path}'")

    # Gộp, loại bỏ trùng lặp và lưu file RECRAWL tổng
    if all_recrawl_dfs:
        master_recrawl_df = pd.concat(all_recrawl_dfs, ignore_index=True)
        final_recrawl_df = master_recrawl_df.drop_duplicates(subset=['title', 'acronym'], keep='first')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng cần recrawl (có ghi chú) vào '{aggregated_recrawl_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'cần recrawl' nào. Đã tạo file rỗng '{aggregated_recrawl_path}'")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # 1. Danh sách các file input cần xử lý
    input_csv_files_for_note_check = [
        './src/conference/evaluate/recrawl_batch_4_check.csv',
        './src/conference/evaluate/recrawl_batch_5_check.csv',
        './src/conference/evaluate/recrawl_batch_9_check.csv',
    ]

    # 2. Thư mục để chứa các file output riêng lẻ
    individual_output_directory_note = './src/conference/evaluate/tri_check_outputs'

    # 3. Đường dẫn cho 2 file tổng hợp cuối cùng
    aggregated_correct_file_note = './src/conference/evaluate/tri_all_correct.csv'
    aggregated_recrawl_file_note = './src/conference/evaluate/tri_all_recrawl.csv'

    # 4. Gọi hàm xử lý chính
    process_files_by_note(
        input_files=input_csv_files_for_note_check,
        output_dir=individual_output_directory_note,
        aggregated_correct_path=aggregated_correct_file_note,
        aggregated_recrawl_path=aggregated_recrawl_file_note
    )


--- Đang xử lý file: ./src/conference/evaluate/recrawl_batch_4_check.csv ---
Đã đọc thành công 26 dòng.
Cảnh báo: Không tìm thấy cột 'Note'. Coi như tất cả các dòng đều 'đúng'.
Kết quả tách: 26 dòng không có ghi chú (đúng), 0 dòng có ghi chú (cần recrawl).
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tri_check_outputs'

--- Đang xử lý file: ./src/conference/evaluate/recrawl_batch_5_check.csv ---
Đã đọc thành công 27 dòng.
Cảnh báo: Không tìm thấy cột 'Note'. Coi như tất cả các dòng đều 'đúng'.
Kết quả tách: 27 dòng không có ghi chú (đúng), 0 dòng có ghi chú (cần recrawl).
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tri_check_outputs'

--- Đang xử lý file: ./src/conference/evaluate/recrawl_batch_9_check.csv ---
Đã đọc thành công 24 dòng.
Cảnh báo: Không tìm thấy cột 'Note'. Coi như tất cả các dòng đều 'đúng'.
Kết quả tách: 24 dòng không có ghi chú (đúng), 0 dòng có ghi chú (cần recrawl).
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tr

In [15]:
import csv

# --- Cấu hình tên file ---
file_danh_gia = './src/conference/evaluate/individual_outputs/batch12_recrawl.csv'
file_core_2023 = './src/conference/csv/CORE_2023.csv'
output_file = './src/conference/evaluate/file_to_recrawl_batch_12_lan_1.csv'

# --- Bước 1: Đọc file "đánh giá" và tạo một set các khóa (title, acronym) để tra cứu ---
# Sử dụng set để tra cứu nhanh hơn rất nhiều so với list
lookup_keys = set()

try:
    # Mở file với encoding='utf-8' để đảm bảo đọc được tiếng Việt
    with open(file_danh_gia, mode='r', newline='', encoding='utf-8') as f_danhgia:
        # Dùng DictReader để dễ dàng truy cập cột bằng tên header
        reader = csv.DictReader(f_danhgia)
        for row in reader:
            # Lấy title và acronym, loại bỏ khoảng trắng thừa ở đầu và cuối
            title = row.get('title', '').strip()
            acronym = row.get('acronym', '').strip()
            
            # Chỉ thêm vào set nếu cả title và acronym đều có giá trị
            if title and acronym:
                lookup_keys.add((title, acronym))
    print(f"Đã tìm thấy {len(lookup_keys)} khóa (title, acronym) duy nhất từ file '{file_danh_gia}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_danh_gia}'. Vui lòng kiểm tra lại tên và đường dẫn file.")
    exit()


# --- Bước 2: Đọc file CORE_2023, lọc và ghi ra file mới ---
matching_rows = []

try:
    with open(file_core_2023, mode='r', newline='', encoding='utf-8') as f_core:
        # File CORE không có header, dùng csv.reader bình thường
        reader = csv.reader(f_core)
        for row in reader:
            # Đảm bảo dòng có đủ cột để tránh lỗi IndexError
            if len(row) > 2:
                # Cột 2 là title (index 1), Cột 3 là acronym (index 2)
                core_title = row[1].strip()
                core_acronym = row[2].strip()
                
                # Tạo khóa từ file CORE
                current_key = (core_title, core_acronym)
                
                # Kiểm tra xem khóa này có trong set đã tạo ở bước 1 không
                if current_key in lookup_keys:
                    matching_rows.append(row)

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_core_2023}'. Vui lòng kiểm tra lại tên và đường dẫn file.")
    exit()


# --- Bước 3: Ghi các dòng đã lọc vào file output ---
if matching_rows:
    with open(output_file, mode='w', newline='', encoding='utf-8') as f_output:
        writer = csv.writer(f_output)
        writer.writerows(matching_rows)
    print(f"Đã lọc và ghi {len(matching_rows)} conference vào file '{output_file}'.")
else:
    print("Không tìm thấy conference nào trùng khớp.")

Đã tìm thấy 19 khóa (title, acronym) duy nhất từ file './src/conference/evaluate/individual_outputs/batch12_recrawl.csv'.
Đã lọc và ghi 16 conference vào file './src/conference/evaluate/file_to_recrawl_batch_12_lan_1.csv'.


#### EXTRACT

1. Add Extract System Instruction Prefix

In [63]:
import csv
import os

# Định nghĩa các giới hạn độ dài để dễ thay đổi
GIOI_HAN_INPUT = 40000
GIOI_HAN_OUTPUT = 5000
DO_DAI_CHUOI_NGAN = 80 # Số ký tự hiển thị khi báo lỗi

def them_system_instruction_va_prefix_va_loc_csv(duong_dan_file_csv_input, duong_dan_file_csv_output, duong_dan_file_txt_instruction):
    """
    Đọc nội dung từ file instruction.txt.
    Đọc file CSV, thêm nội dung instruction.txt, sau đó thêm dòng "Page content:"
    và cuối cùng mới tới nội dung gốc của cột 'input:'.
    Kiểm tra độ dài của cột 'input:' (sau khi thêm instruction và prefix).
    Nếu bất kỳ dòng nào có cột 'input:' vượt quá GIOI_HAN_INPUT
    hoặc cột 'output:' vượt quá GIOI_HAN_OUTPUT, dòng đó sẽ bị bỏ qua.
    Các dòng hợp lệ (với instruction và prefix đã thêm) sẽ được ghi vào một file CSV mới.

    Args:
        duong_dan_file_csv_input (str): Đường dẫn đến file CSV gốc.
        duong_dan_file_csv_output (str): Đường dẫn đến file CSV mới sẽ được tạo.
        duong_dan_file_txt_instruction (str): Đường dẫn đến file .txt chứa system instruction.

    Returns:
        tuple: (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi, co_loi_doc_instruction).
               Trả về (-1, -1, -1, True) nếu có lỗi nghiêm trọng.
    """

    print(f"--- Bắt đầu xử lý file: {duong_dan_file_csv_input} ---")
    so_dong_goc = 0
    so_dong_vuot_gioi_han = 0
    so_dong_ghi_ra_file_moi = 0
    cac_loi_xu_ly_dong = []
    system_instruction = ""
    co_loi_doc_instruction = False
    prefix_content = "" # Chuỗi sẽ được thêm vào sau instruction

    # Bước 1: Đọc nội dung từ file instruction.txt
    try:
        with open(duong_dan_file_txt_instruction, 'r', encoding='utf-8') as file_instruction:
            system_instruction = file_instruction.read()
        print(f"Đã đọc thành công system instruction từ '{duong_dan_file_txt_instruction}' (độ dài: {len(system_instruction)})")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file instruction tại '{duong_dan_file_txt_instruction}'.")
        co_loi_doc_instruction = True
        # Tiếp tục xử lý CSV nhưng với instruction rỗng
    except Exception as e:
        print(f"Lỗi không xác định khi đọc file instruction: {e}")
        co_loi_doc_instruction = True
        # Tiếp tục xử lý CSV nhưng với instruction rỗng

    # Bước 2: Xử lý file CSV
    try:
        with open(duong_dan_file_csv_input, 'r', encoding='utf-8') as file_csv_in:
            doc_csv = csv.DictReader(file_csv_in)
            ten_cac_cot = doc_csv.fieldnames

            if not ten_cac_cot:
                 print("Lỗi: File CSV trống hoặc không có header.")
                 return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

            if 'input:' not in ten_cac_cot or 'output:' not in ten_cac_cot:
                 print("Lỗi: File CSV phải có cả cột 'input:' và 'output:'.")
                 return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

            print(f"Các cột trong file: {', '.join(ten_cac_cot)}")
            print(f"Giới hạn độ dài kiểm tra (sau khi thêm instruction và prefix): input: > {GIOI_HAN_INPUT}, output: > {GIOI_HAN_OUTPUT}")
            print(f"File đầu ra sẽ là: {duong_dan_file_csv_output}")
            print("-" * 30)

            # Tạo thư mục chứa file output nếu chưa tồn tại
            thu_muc_output = os.path.dirname(duong_dan_file_csv_output)
            if thu_muc_output and not os.path.exists(thu_muc_output):
                os.makedirs(thu_muc_output)
                print(f"Đã tạo thư mục đầu ra: {thu_muc_output}")

            with open(duong_dan_file_csv_output, 'w', encoding='utf-8', newline='') as file_csv_out:
                ghi_csv = csv.DictWriter(file_csv_out, fieldnames=ten_cac_cot)
                ghi_csv.writeheader() # Ghi header vào file mới

                for so_dong_hien_tai, dong in enumerate(doc_csv, start=2): # Bắt đầu từ dòng 2 (sau header)
                    so_dong_goc += 1
                    vuot_gioi_han = False
                    chi_tiet_vuot = [] # Lưu chi tiết cột nào vượt giới hạn

                    try:
                        # Lấy giá trị input gốc và thêm instruction + prefix
                        gia_tri_input_goc = dong.get('input:', '') # Sử dụng get với giá trị mặc định
                        # Nối theo thứ tự: instruction -> prefix -> input gốc
                        gia_tri_input_moi = system_instruction + prefix_content + gia_tri_input_goc

                        # Cập nhật giá trị input trong dictionary của dòng
                        dong['input:'] = gia_tri_input_moi
                        do_dai_input_moi = len(gia_tri_input_moi)

                        # Kiểm tra độ dài input mới
                        if do_dai_input_moi > GIOI_HAN_INPUT:
                            vuot_gioi_han = True
                            chi_tiet_vuot.append(f"input: ({do_dai_input_moi}/{GIOI_HAN_INPUT})")

                        # Kiểm tra độ dài output (không thay đổi)
                        gia_tri_output = dong.get('output:', '')
                        do_dai_output = len(gia_tri_output)
                        if do_dai_output > GIOI_HAN_OUTPUT:
                            vuot_gioi_han = True
                            chi_tiet_vuot.append(f"output: ({do_dai_output}/{GIOI_HAN_OUTPUT})")


                        if vuot_gioi_han:
                            so_dong_vuot_gioi_han += 1
                            print(f"Dòng {so_dong_hien_tai}: Bỏ qua vì vượt giới hạn - {' & '.join(chi_tiet_vuot)}")
                        else:
                            # Dòng hợp lệ, ghi vào file mới (với giá trị input đã cập nhật)
                            ghi_csv.writerow(dong)
                            so_dong_ghi_ra_file_moi += 1

                    except Exception as e:
                         # Xử lý lỗi đọc hoặc ghi dòng cụ thể
                         print(f"Lỗi không xác định khi xử lý dòng {so_dong_hien_tai}: {e}")
                         cac_loi_xu_ly_dong.append(f"Dòng {so_dong_hien_tai}: {e}")
                         # Không tăng so_dong_vuot_gioi_han hoặc so_dong_ghi_ra_file_moi cho dòng lỗi này

            print("-" * 30)
            print("--- Kết thúc xử lý file CSV ---")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv_input}'.")
        return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

    except Exception as e:
        print(f"Lỗi không xác định trong quá trình xử lý file CSV: {e}")
        return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

    if cac_loi_xu_ly_dong:
        print("\nCác lỗi xử lý dòng cụ thể:")
        for loi in cac_loi_xu_ly_dong:
            print(f"- {loi}")

    return (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi, co_loi_doc_instruction)


# --- Phần ví dụ sử dụng ---
duong_dan_file_goc = "./src/conference/train/extract_infor_train_data_formatted_filtered.csv"       # Thay đổi đường dẫn file CSV gốc của bạn
duong_dan_file_moi = "./src/conference/train/extract_infor_train_data_formatted_filtered_with_instruction_prefix.csv" # Tên file mới sẽ được tạo
duong_dan_instruction = "extract_system_instruction.txt"     # Thay đổi đường dẫn file instruction.txt

so_dong_goc, so_dong_vuot, so_dong_moi, co_loi_instruction = them_system_instruction_va_prefix_va_loc_csv(
    duong_dan_file_goc,
    duong_dan_file_moi,
    duong_dan_instruction
)

print("\n--- Báo cáo kết quả xử lý ---")
if co_loi_instruction:
    print("Cảnh báo: Có lỗi khi đọc file system instruction. Quá trình xử lý CSV tiếp tục nhưng không thêm instruction.")

if so_dong_goc != -1:
    print(f"Tổng số dòng trong file gốc (không bao gồm header): {so_dong_goc}")
    print(f"Số dòng bị loại bỏ (vượt quá giới hạn sau khi thêm instruction và prefix): {so_dong_vuot}")
    print(f"Số dòng được ghi vào file mới '{duong_dan_file_moi}': {so_dong_moi}")
else:
    print("\nKhông thể tạo báo cáo chi tiết do có lỗi nghiêm trọng trong quá trình xử lý.")

--- Bắt đầu xử lý file: ./src/conference/train/extract_infor_train_data_formatted_filtered.csv ---
Đã đọc thành công system instruction từ 'extract_system_instruction.txt' (độ dài: 3832)
Các cột trong file: input:, output:
Giới hạn độ dài kiểm tra (sau khi thêm instruction và prefix): input: > 40000, output: > 5000
File đầu ra sẽ là: ./src/conference/train/extract_infor_train_data_formatted_filtered_with_instruction_prefix.csv
------------------------------
Dòng 127: Bỏ qua vì vượt giới hạn - input: (41137/40000)
Dòng 143: Bỏ qua vì vượt giới hạn - input: (42952/40000)
Dòng 178: Bỏ qua vì vượt giới hạn - input: (42386/40000)
Dòng 231: Bỏ qua vì vượt giới hạn - input: (42758/40000)
Dòng 241: Bỏ qua vì vượt giới hạn - input: (40726/40000)
Dòng 255: Bỏ qua vì vượt giới hạn - input: (40652/40000)
------------------------------
--- Kết thúc xử lý file CSV ---

--- Báo cáo kết quả xử lý ---
Tổng số dòng trong file gốc (không bao gồm header): 282
Số dòng bị loại bỏ (vượt quá giới hạn sau khi 

1. Add Determine System Instruction Prefix

In [58]:
import csv
import os

# --- Định nghĩa các biến năm tùy chỉnh ---
NAM_HIEN_TAI = 2025  # Năm bạn muốn ưu tiên tìm kiếm
NAM_TRUOC_DO = 2024  # Năm dự phòng nếu năm hiện tại không có

# Định nghĩa các giới hạn độ dài để dễ thay đổi
GIOI_HAN_INPUT = 40000
GIOI_HAN_OUTPUT = 5000
DO_DAI_CHUOI_NGAN = 80 # Số ký tự hiển thị khi báo lỗi cho đoạn vượt quá

def them_system_instruction_va_prefix_va_loc_csv(duong_dan_file_csv_input, duong_dan_file_csv_output, duong_dan_file_txt_instruction, nam_hien_tai_config, nam_truoc_do_config):
    """
    Đọc nội dung từ file instruction.txt.
    Đọc file CSV, thêm nội dung instruction.txt, sau đó thêm dòng "Page content:"
    và cuối cùng mới tới nội dung gốc của cột 'input:'.
    Kiểm tra độ dài của cột 'input:' (sau khi thêm instruction và prefix).
    Nếu bất kỳ dòng nào có cột 'input:' vượt quá GIOI_HAN_INPUT
    hoặc cột 'output:' vượt quá GIOI_HAN_OUTPUT, dòng đó sẽ bị bỏ qua.
    Các dòng hợp lệ (với instruction và prefix đã thêm) sẽ được ghi vào một file CSV mới.

    Args:
        duong_dan_file_csv_input (str): Đường dẫn đến file CSV gốc.
        duong_dan_file_csv_output (str): Đường dẫn đến file CSV mới sẽ được tạo.
        duong_dan_file_txt_instruction (str): Đường dẫn đến file .txt chứa system instruction.
        nam_hien_tai_config (int): Năm ưu tiên tìm kiếm (ví dụ: 2025).
        nam_truoc_do_config (int): Năm dự phòng nếu năm hiện tại không có (ví dụ: 2024).

    Returns:
        tuple: (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi, co_loi_doc_instruction).
               Trả về (-1, -1, -1, True) nếu có lỗi nghiêm trọng.
    """

    print(f"--- Bắt đầu xử lý file: {duong_dan_file_csv_input} ---")
    so_dong_goc = 0
    so_dong_vuot_gioi_han = 0
    so_dong_ghi_ra_file_moi = 0
    cac_loi_xu_ly_dong = []
    system_instruction = ""
    co_loi_doc_instruction = False
    prefix_content = "\n\n" # Chuỗi sẽ được thêm vào sau instruction

    # Bước 1: Đọc nội dung từ file instruction.txt
    try:
        with open(duong_dan_file_txt_instruction, 'r', encoding='utf-8') as file_instruction:
            system_instruction_raw = file_instruction.read()
            # Thay thế năm trong instruction
            system_instruction = system_instruction_raw.replace("2025", str(nam_hien_tai_config))
            system_instruction = system_instruction.replace("2024", str(nam_truoc_do_config))

        print(f"Đã đọc thành công system instruction từ '{duong_dan_file_txt_instruction}' (độ dài sau thay thế: {len(system_instruction)})")
        print(f"Năm được sử dụng trong instruction: Ưu tiên {nam_hien_tai_config}, dự phòng {nam_truoc_do_config}")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file instruction tại '{duong_dan_file_txt_instruction}'.")
        co_loi_doc_instruction = True
        # Tiếp tục xử lý CSV nhưng với instruction rỗng
    except Exception as e:
        print(f"Lỗi không xác định khi đọc file instruction: {e}")
        co_loi_doc_instruction = True
        # Tiếp tục xử lý CSV nhưng với instruction rỗng

    # Bước 2: Xử lý file CSV
    try:
        with open(duong_dan_file_csv_input, 'r', encoding='utf-8') as file_csv_in:
            doc_csv = csv.DictReader(file_csv_in)
            ten_cac_cot = doc_csv.fieldnames

            if not ten_cac_cot:
                 print("Lỗi: File CSV trống hoặc không có header.")
                 return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

            if 'input:' not in ten_cac_cot or 'output:' not in ten_cac_cot:
                 print("Lỗi: File CSV phải có cả cột 'input:' và 'output:'.")
                 return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

            print(f"Các cột trong file: {', '.join(ten_cac_cot)}")
            print(f"Giới hạn độ dài kiểm tra (sau khi thêm instruction và prefix): input: > {GIOI_HAN_INPUT}, output: > {GIOI_HAN_OUTPUT}")
            print(f"File đầu ra sẽ là: {duong_dan_file_csv_output}")
            print("-" * 30)

            # Tạo thư mục chứa file output nếu chưa tồn tại
            thu_muc_output = os.path.dirname(duong_dan_file_csv_output)
            if thu_muc_output and not os.path.exists(thu_muc_output):
                os.makedirs(thu_muc_output)
                print(f"Đã tạo thư mục đầu ra: {thu_muc_output}")

            with open(duong_dan_file_csv_output, 'w', encoding='utf-8', newline='') as file_csv_out:
                ghi_csv = csv.DictWriter(file_csv_out, fieldnames=ten_cac_cot)
                ghi_csv.writeheader() # Ghi header vào file mới

                for so_dong_hien_tai, dong in enumerate(doc_csv, start=2): # Bắt đầu từ dòng 2 (sau header)
                    so_dong_goc += 1
                    vuot_gioi_han = False
                    chi_tiet_vuot = [] # Lưu chi tiết cột nào vượt giới hạn
                    doan_vuot_input = "" # Lưu đoạn input vượt giới hạn
                    doan_vuot_output = "" # Lưu đoạn output vượt giới hạn

                    try:
                        # Lấy giá trị input gốc và thêm instruction + prefix
                        gia_tri_input_goc = dong.get('input:', '') # Sử dụng get với giá trị mặc định
                        # Nối theo thứ tự: instruction -> prefix -> input gốc
                        gia_tri_input_moi = system_instruction + prefix_content + gia_tri_input_goc

                        # Cập nhật giá trị input trong dictionary của dòng
                        dong['input:'] = gia_tri_input_moi
                        do_dai_input_moi = len(gia_tri_input_moi)

                        # Kiểm tra độ dài input mới
                        if do_dai_input_moi > GIOI_HAN_INPUT:
                            vuot_gioi_han = True
                            chi_tiet_vuot.append(f"input: ({do_dai_input_moi}/{GIOI_HAN_INPUT})")
                            # Lấy một đoạn chuỗi để kiểm tra
                            doan_vuot_input = gia_tri_input_moi[GIOI_HAN_INPUT - DO_DAI_CHUOI_NGAN : GIOI_HAN_INPUT + DO_DAI_CHUOI_NGAN]
                            if doan_vuot_input: # Thêm dấu ba chấm nếu chuỗi bị cắt
                                doan_vuot_input = f"...{doan_vuot_input}..."


                        # Kiểm tra độ dài output (không thay đổi)
                        gia_tri_output = dong.get('output:', '')
                        do_dai_output = len(gia_tri_output)
                        if do_dai_output > GIOI_HAN_OUTPUT:
                            vuot_gioi_han = True
                            chi_tiet_vuot.append(f"output: ({do_dai_output}/{GIOI_HAN_OUTPUT})")
                            # Lấy một đoạn chuỗi để kiểm tra
                            doan_vuot_output = gia_tri_output[GIOI_HAN_OUTPUT - DO_DAI_CHUOI_NGAN : GIOI_HAN_OUTPUT + DO_DAI_CHUOI_NGAN]
                            if doan_vuot_output: # Thêm dấu ba chấm nếu chuỗi bị cắt
                                doan_vuot_output = f"...{doan_vuot_output}..."


                        if vuot_gioi_han:
                            so_dong_vuot_gioi_han += 1
                            print(f"Dòng {so_dong_hien_tai}: Bỏ qua vì vượt giới hạn - {' & '.join(chi_tiet_vuot)}")
                            if doan_vuot_input:
                                print(f"    [Đoạn input vượt giới hạn]: '{doan_vuot_input}'")
                            if doan_vuot_output:
                                print(f"    [Đoạn output vượt giới hạn]: '{doan_vuot_output}'")
                        else:
                            # Dòng hợp lệ, ghi vào file mới (với giá trị input đã cập nhật)
                            ghi_csv.writerow(dong)
                            so_dong_ghi_ra_file_moi += 1

                    except Exception as e:
                         # Xử lý lỗi đọc hoặc ghi dòng cụ thể
                         print(f"Lỗi không xác định khi xử lý dòng {so_dong_hien_tai}: {e}")
                         cac_loi_xu_ly_dong.append(f"Dòng {so_dong_hien_tai}: {e}")
                         # Không tăng so_dong_vuot_gioi_han hoặc so_dong_ghi_ra_file_moi cho dòng lỗi này

            print("-" * 30)
            print("--- Kết thúc xử lý file CSV ---")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv_input}'.")
        return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

    except Exception as e:
        print(f"Lỗi không xác định trong quá trình xử lý file CSV: {e}")
        return (-1, -1, -1, False) # Trả về lỗi nghiêm trọng

    if cac_loi_xu_ly_dong:
        print("\nCác lỗi xử lý dòng cụ thể:")
        for loi in cac_loi_xu_ly_dong:
            print(f"- {loi}")

    return (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi, co_loi_doc_instruction)


# --- Phần ví dụ sử dụng ---
duong_dan_file_goc = "./src/conference/train/determine_links_train_data_formatted_filtered.csv"       # Thay đổi đường dẫn file CSV gốc của bạn
duong_dan_file_moi = "./src/conference/train/determine_links_train_data_formatted_filtered_with_instruction_prefix.csv" # Tên file mới sẽ được tạo
duong_dan_instruction = "determine_system_instruction.txt"     # Thay đổi đường dẫn file instruction.txt

so_dong_goc, so_dong_vuot, so_dong_moi, co_loi_instruction = them_system_instruction_va_prefix_va_loc_csv(
    duong_dan_file_goc,
    duong_dan_file_moi,
    duong_dan_instruction,
    NAM_HIEN_TAI,  # Truyền biến năm vào hàm
    NAM_TRUOC_DO   # Truyền biến năm vào hàm
)

print("\n--- Báo cáo kết quả xử lý ---")
if co_loi_instruction:
    print("Cảnh báo: Có lỗi khi đọc file system instruction. Quá trình xử lý CSV tiếp tục nhưng không thêm instruction.")

if so_dong_goc != -1:
    print(f"Tổng số dòng trong file gốc (không bao gồm header): {so_dong_goc}")
    print(f"Số dòng bị loại bỏ (vượt quá giới hạn sau khi thêm instruction và prefix): {so_dong_vuot}")
    print(f"Số dòng được ghi vào file mới '{duong_dan_file_moi}': {so_dong_moi}")
else:
    print("\nKhông thể tạo báo cáo chi tiết do có lỗi nghiêm trọng trong quá trình xử lý.")

--- Bắt đầu xử lý file: ./src/conference/train/determine_links_train_data_formatted_filtered.csv ---
Đã đọc thành công system instruction từ 'determine_system_instruction.txt' (độ dài sau thay thế: 5291)
Năm được sử dụng trong instruction: Ưu tiên 2025, dự phòng 2024
Các cột trong file: input:, output:
Giới hạn độ dài kiểm tra (sau khi thêm instruction và prefix): input: > 40000, output: > 5000
File đầu ra sẽ là: ./src/conference/train/determine_links_train_data_formatted_filtered_with_instruction_prefix.csv
------------------------------
Dòng 118: Bỏ qua vì vượt giới hạn - input: (40395/40000)
    [Đoạn input vượt giới hạn]: '...ef="/cfp/about.jsp" - About Us
|href="mailto:wikicfp@gmail.com" - Contact Us
|href="/cfp/data.jsp" - Data
|href="/cfp/privacy.jsp" - Privacy Policy
|href="/cfp/...'
Dòng 148: Bỏ qua vì vượt giới hạn - input: (43769/40000)
    [Đoạn input vượt giới hạn]: '...e=WikiCFP&utm_medium=Display&utm_term=home&utm_content=semantic-scholar-rail-try-it&utm_campaign=WikiCFP

In [ ]:
# import csv
# import json
# import io
# import sys

# # --- Phần 1: Tăng giới hạn kích thước trường CSV ---
# max_int = sys.maxsize
# print("Đang tìm giới hạn kích thước trường CSV tối đa mà hệ thống hỗ trợ...")
# while True:
#     try:
#         csv.field_size_limit(max_int)
#         print(f"Đã đặt giới hạn kích thước trường CSV thành: {max_int}")
#         break
#     except OverflowError:
#         max_int = int(max_int / 10)

# # --- Phần 2: Đặt tên file và xử lý ---
# input_filename = './src/conference/examples/determine_links_train_data.csv' # Tên file CSV đầu vào của bạn
# # Đổi tên file output một lần nữa
# output_filename = 'output_transformed_final.csv'

# print(f"Đang xử lý file: {input_filename}")

# try:
#     with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
#          open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

#         csv_reader = csv.reader(infile)
#         csv_writer = csv.writer(outfile)

#         try:
#             header = next(csv_reader)
#             csv_writer.writerow(header)
#             print(f"Đã đọc header: {header}")

#             for i, row in enumerate(csv_reader):
#                 current_line_num = i + 2
#                 if len(row) == 2:
#                     input_text = row[0]
#                     output_json_str = row[1]
#                     transformed_output = ""

#                     try:
#                         data_dict = json.loads(output_json_str)

#                         if isinstance(data_dict, dict):
#                             # ---- THAY ĐỔI Ở ĐÂY ----
#                             lines = [] # Danh sách để lưu các dòng đã định dạng
#                             num_items = len(data_dict) # Lấy số lượng cặp key-value

#                             # Sử dụng enumerate để có chỉ số (idx)
#                             for idx, (key, value) in enumerate(data_dict.items()):
#                                 # Tạo dòng cơ bản
#                                 line = f'"{key}": "{value}"'
#                                 # Nếu không phải là mục cuối cùng, thêm dấu phẩy
#                                 if idx < num_items - 1:
#                                     line += ","
#                                 lines.append(line) # Thêm dòng vào danh sách

#                             # Nối các dòng trong danh sách bằng ký tự xuống dòng
#                             transformed_output = "\n".join(lines)
#                             # ---- KẾT THÚC THAY ĐỔI ----
#                         else:
#                             print(f"Cảnh báo dòng {current_line_num}: Cột output không phải là JSON object: {output_json_str[:100]}...")
#                             transformed_output = output_json_str

#                     except json.JSONDecodeError:
#                         print(f"Cảnh báo dòng {current_line_num}: Không thể parse JSON từ cột output: {output_json_str[:100]}...")
#                         transformed_output = output_json_str
#                     except Exception as e:
#                         print(f"Lỗi xử lý JSON ở dòng {current_line_num}: {e}")
#                         transformed_output = output_json_str

#                     csv_writer.writerow([input_text, transformed_output])

#                 else:
#                     print(f"Cảnh báo dòng {current_line_num}: Dòng không có đúng 2 cột. Bỏ qua: {row}")

#         except csv.Error as e:
#             print(f"Lỗi CSV ở dòng {csv_reader.line_num}: {e}")
#             print("Kiểm tra lại cấu trúc file CSV hoặc tăng 'field_size_limit' nếu cần.")
#         except StopIteration:
#              print("File CSV không có dữ liệu sau header.")
#         except Exception as e:
#              line_num_info = f" (possibly near line {csv_reader.line_num})" if 'csv_reader' in locals() and hasattr(csv_reader, 'line_num') else ""
#              print(f"Đã xảy ra lỗi không mong muốn trong quá trình xử lý file{line_num_info}: {e}")

#     print(f"\nXử lý hoàn tất. Dữ liệu đã được ghi vào file: {output_filename}")

# except FileNotFoundError:
#     print(f"Lỗi: Không tìm thấy file '{input_filename}'")
# except Exception as e:
#     print(f"Đã xảy ra lỗi không mong muốn bên ngoài quá trình đọc CSV: {e}")

In [ ]:
# import csv
# import re
# import sys

# # --- Tăng giới hạn CSV nếu cần ---
# max_int = sys.maxsize
# print("Đang tìm giới hạn kích thước trường CSV tối đa...")
# while True:
#     try:
#         csv.field_size_limit(max_int)
#         print(f"Đã đặt giới hạn kích thước trường CSV thành: {max_int}")
#         break
#     except OverflowError:
#         max_int = int(max_int / 10)

# # --- Đặt tên file ---
# input_filename = 'output_transformed_final.csv' # File input từ bước trước
# # File output mới, tên rõ ràng hơn
# output_filename = 'final_truncated_input_at_4.csv'

# # --- Biểu thức chính quy để tìm điểm cắt (CHỈ SỐ 4) ---
# # Thay đổi ở đây: thay \d+ bằng số 4 cụ thể
# pattern_to_find = re.compile(r"\s*4\.\s+Website of ") # Tìm "4. Website of "

# print(f"Đang đọc file: {input_filename}")
# print(f"Sẽ ghi kết quả vào: {output_filename}")

# try:
#     with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
#          open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

#         csv_reader = csv.reader(infile)
#         csv_writer = csv.writer(outfile)

#         try:
#             # Đọc và ghi header
#             header = next(csv_reader)
#             csv_writer.writerow(header)
#             print(f"Đã đọc header: {header}")

#             # Xử lý từng dòng dữ liệu CSV
#             for i, row in enumerate(csv_reader):
#                 current_csv_line_num = i + 2 # Số dòng trong file CSV

#                 if len(row) == 2:
#                     input_text_original = row[0]
#                     output_text = row[1] # Giữ nguyên cột output

#                     # Tìm vị trí đầu tiên của pattern "4. Website of "
#                     match = pattern_to_find.search(input_text_original)

#                     if match:
#                         # Nếu tìm thấy, lấy vị trí bắt đầu
#                         start_index = match.start()
#                         # Cắt chuỗi từ đầu đến vị trí đó
#                         input_text_cleaned = input_text_original[:start_index].rstrip()
#                         # Thêm dấu xuống hàng ở cuối sau khi cắt
#                         input_text_cleaned += '\n'
#                         # print(f"  (CSV Line {current_csv_line_num}) Đã cắt bỏ từ '4. Website of ' tại vị trí {start_index}.")
#                     else:
#                         # Nếu không tìm thấy "4. Website of ", giữ nguyên và thêm dấu xuống hàng
#                         input_text_cleaned = input_text_original + '\n'


#                     # Ghi dòng đã xử lý
#                     csv_writer.writerow([input_text_cleaned, output_text])

#                 else:
#                     print(f"Cảnh báo dòng CSV {current_csv_line_num}: Dòng không có đúng 2 cột. Bỏ qua: {row}")

#         except csv.Error as e:
#             print(f"Lỗi CSV ở dòng {csv_reader.line_num}: {e}")
#         except StopIteration:
#              print("File CSV không có dữ liệu sau header.")
#         except Exception as e:
#              line_num_info = f" (possibly near line {csv_reader.line_num})" if 'csv_reader' in locals() and hasattr(csv_reader, 'line_num') else ""
#              print(f"Đã xảy ra lỗi không mong muốn trong quá trình xử lý file{line_num_info}: {e}")

#     print(f"\nXử lý hoàn tất. Dữ liệu đã được cắt (chỉ tại '4. Website of ') và ghi vào file: {output_filename}")

# except FileNotFoundError:
#     print(f"Lỗi: Không tìm thấy file '{input_filename}'. Hãy đảm bảo file input có tên này.")
# except Exception as e:
#     print(f"Đã xảy ra lỗi không mong muốn bên ngoài quá trình đọc CSV: {e}")

2. Check giới hạn 40000 và 5000

In [11]:
import csv
import os

# Định nghĩa các giới hạn độ dài để dễ thay đổi
GIOI_HAN_INPUT = 40000
GIOI_HAN_OUTPUT = 5000
DO_DAI_CHUOI_NGAN = 80 # Số ký tự hiển thị khi báo lỗi

def xoa_dong_vuot_gioi_han_csv(duong_dan_file_csv_input, duong_dan_file_csv_output):
    """
    Đọc file CSV, kiểm tra độ dài của cột 'input:' và 'output:'.
    Nếu bất kỳ dòng nào có cột 'input:' vượt quá GIOI_HAN_INPUT
    hoặc cột 'output:' vượt quá GIOI_HAN_OUTPUT, dòng đó sẽ bị bỏ qua.
    Các dòng hợp lệ sẽ được ghi vào một file CSV mới.

    Args:
        duong_dan_file_csv_input (str): Đường dẫn đến file CSV gốc.
        duong_dan_file_csv_output (str): Đường dẫn đến file CSV mới sẽ được tạo.

    Returns:
        tuple: (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi).
               Trả về (-1, -1, -1) nếu có lỗi xảy ra.
    """

    print(f"--- Bắt đầu xử lý file: {duong_dan_file_csv_input} ---")
    so_dong_goc = 0
    so_dong_vuot_gioi_han = 0
    so_dong_ghi_ra_file_moi = 0
    cac_loi_xu_ly = []

    try:
        with open(duong_dan_file_csv_input, 'r', encoding='utf-8') as file_csv_in:
            doc_csv = csv.DictReader(file_csv_in)
            ten_cac_cot = doc_csv.fieldnames

            if not ten_cac_cot:
                 print("Lỗi: File CSV trống hoặc không có header.")
                 return (-1, -1, -1)

            print(f"Các cột trong file: {', '.join(ten_cac_cot)}")
            print(f"Giới hạn độ dài kiểm tra: input: > {GIOI_HAN_INPUT}, output: > {GIOI_HAN_OUTPUT}")
            print(f"File đầu ra sẽ là: {duong_dan_file_csv_output}")
            print("-" * 30)

            # Tạo thư mục chứa file output nếu chưa tồn tại
            thu_muc_output = os.path.dirname(duong_dan_file_csv_output)
            if thu_muc_output and not os.path.exists(thu_muc_output):
                os.makedirs(thu_muc_output)
                print(f"Đã tạo thư mục đầu ra: {thu_muc_output}")


            with open(duong_dan_file_csv_output, 'w', encoding='utf-8', newline='') as file_csv_out:
                ghi_csv = csv.DictWriter(file_csv_out, fieldnames=ten_cac_cot)
                ghi_csv.writeheader() # Ghi header vào file mới

                for so_dong_hien_tai, dong in enumerate(doc_csv, start=2): # Bắt đầu từ dòng 2 (sau header)
                    so_dong_goc += 1
                    vuot_gioi_han = False
                    chi_tiet_vuot = [] # Lưu chi tiết cột nào vượt giới hạn

                    try:
                        # Kiểm tra từng cột cần giới hạn
                        if 'input:' in dong:
                            gia_tri_input = dong.get('input:')
                            do_dai_input = len(gia_tri_input) if gia_tri_input is not None else 0
                            if do_dai_input > GIOI_HAN_INPUT:
                                vuot_gioi_han = True
                                chi_tiet_vuot.append(f"input: ({do_dai_input}/{GIOI_HAN_INPUT})")

                        if 'output:' in dong:
                            gia_tri_output = dong.get('output:')
                            do_dai_output = len(gia_tri_output) if gia_tri_output is not None else 0
                            if do_dai_output > GIOI_HAN_OUTPUT:
                                vuot_gioi_han = True
                                chi_tiet_vuot.append(f"output: ({do_dai_output}/{GIOI_HAN_OUTPUT})")

                        if vuot_gioi_han:
                            so_dong_vuot_gioi_han += 1
                            print(f"Dòng {so_dong_hien_tai}: Bỏ qua vì vượt giới hạn - {' & '.join(chi_tiet_vuot)}")
                        else:
                            # Dòng hợp lệ, ghi vào file mới
                            ghi_csv.writerow(dong)
                            so_dong_ghi_ra_file_moi += 1

                    except Exception as e:
                         # Xử lý lỗi đọc hoặc ghi dòng cụ thể
                         print(f"Lỗi không xác định khi xử lý dòng {so_dong_hien_tai}: {e}")
                         cac_loi_xu_ly.append(f"Dòng {so_dong_hien_tai}: {e}")
                         # Không tăng so_dong_vuot_gioi_han hoặc so_dong_ghi_ra_file_moi cho dòng lỗi này

            print("-" * 30)
            print("--- Kết thúc xử lý file ---")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv_input}'.")
        return (-1, -1, -1)

    except Exception as e:
        print(f"Lỗi không xác định trong quá trình xử lý file: {e}")
        return (-1, -1, -1)

    if cac_loi_xu_ly:
        print("\nCác lỗi xử lý dòng cụ thể:")
        for loi in cac_loi_xu_ly:
            print(f"- {loi}")

    return (so_dong_goc, so_dong_vuot_gioi_han, so_dong_ghi_ra_file_moi)


# --- Phần ví dụ sử dụng ---
duong_dan_file_goc = "./src/conference/train/determine_links_train_data_formatted.csv" # Thay đổi đường dẫn file gốc của bạn
duong_dan_file_moi = "./src/conference/train/determine_links_train_data_formatted_filtered.csv" # Tên file mới sẽ được tạo

so_dong_goc, so_dong_vuot, so_dong_moi = xoa_dong_vuot_gioi_han_csv(duong_dan_file_goc, duong_dan_file_moi)

if so_dong_goc != -1:
    print("\n--- Báo cáo kết quả lọc ---")
    print(f"Tổng số dòng trong file gốc (không bao gồm header): {so_dong_goc}")
    print(f"Số dòng bị loại bỏ (vượt quá giới hạn): {so_dong_vuot}")
    print(f"Số dòng được ghi vào file mới '{duong_dan_file_moi}': {so_dong_moi}")
else:
    print("\nKhông thể tạo báo cáo do có lỗi trong quá trình xử lý.")

--- Bắt đầu xử lý file: ./src/conference/train/determine_links_train_data_formatted.csv ---
Các cột trong file: input:, output:
Giới hạn độ dài kiểm tra: input: > 40000, output: > 5000
File đầu ra sẽ là: ./src/conference/train/determine_links_train_data_formatted_filtered.csv
------------------------------
------------------------------
--- Kết thúc xử lý file ---

--- Báo cáo kết quả lọc ---
Tổng số dòng trong file gốc (không bao gồm header): 244
Số dòng bị loại bỏ (vượt quá giới hạn): 0
Số dòng được ghi vào file mới './src/conference/train/determine_links_train_data_formatted_filtered.csv': 244


3. Kiểm tra JSON Output

In [41]:
import csv
import json

def kiem_tra_csv_va_json(duong_dan_file_csv, context_size=50):
    """
    Đọc file CSV, kiểm tra xem cột "output:" có phải là JSON hợp lệ hay không,
    và log dòng lỗi ra cùng với context xung quanh vị trí lỗi.

    Args:
        duong_dan_file_csv (str): Đường dẫn đến file CSV.
        context_size (int): Số lượng ký tự trước và sau vị trí lỗi để hiển thị.

    Returns:
        list: Một danh sách các thông báo lỗi. Mỗi thông báo lỗi là một tuple
              (so_dong, cot, thong_tin_loi). Trả về danh sách rỗng nếu không có lỗi.
    """

    cac_loi = []

    try:
        with open(duong_dan_file_csv, 'r', encoding='utf-8') as file_csv:
            doc_csv = csv.DictReader(file_csv)
            for so_dong, dong in enumerate(doc_csv, start=2):  # Bắt đầu từ dòng 2 (bỏ qua header)
                try:
                    input_text = dong.get('input:')
                    output_text = dong.get('output:')

                    # Kiểm tra JSON
                    if output_text is not None:
                        try:
                            json.loads(output_text)
                        except json.JSONDecodeError as e:
                            # Log dòng lỗi ra và context
                            position = e.pos
                            start = max(0, position - context_size)
                            end = min(len(output_text), position + context_size)
                            context = output_text[start:end]

                            print(f"Lỗi JSON ở dòng {so_dong}: {e}")
                            print(f"Vị trí lỗi: {position}")
                            print(f"Context: ...{context}...")
                            cac_loi.append((so_dong, 'output:', f"Lỗi JSON: {e}"))


                except Exception as e:
                    print(f"Lỗi khi xử lý dòng {so_dong}: {e}")
                    cac_loi.append((so_dong, 'Lỗi xử lý dòng', str(e)))  # Thêm thông tin lỗi vào danh sách

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv}'.")
        return [("File", "Không tìm thấy file", duong_dan_file_csv)]  # Trả về một list với thông báo lỗi

    except Exception as e:
        print(f"Lỗi không xác định khi đọc file CSV: {e}")
        return [("File", "Lỗi đọc file", str(e))]

    return cac_loi


# Ví dụ sử dụng
duong_dan_file = 'D:/NEW-SERVER-TS/src/conference/train/determine_links_train_data_formatted_filtered.csv'  # Thay đổi đường dẫn file của bạn ở đây
cac_loi = kiem_tra_csv_va_json(duong_dan_file)

if cac_loi:
    print("Các lỗi được tìm thấy:")
    for so_dong, cot, thong_tin_loi in cac_loi:
        print(f"  - Dòng {so_dong}, cột '{cot}': {thong_tin_loi}")
else:
    print("Không có lỗi nào được tìm thấy.")

Không có lỗi nào được tìm thấy.


4. Transform determine links

In [ ]:
import csv
import json
import re

def transform_single_input(old_input_text):
    """
    Transforms a single 'input:' string from the old format to the new format.
    """
    new_input_parts = []

    # 1. Extract Conference Info
    conf_title = "Unknown Conference"
    conf_acronym = "N/A" # Default value

    # Try to find full name and acronym in parentheses
    conf_name_match = re.search(
        r"Conference full name:\s*(.*?)(?:\s*\((.*?)\))?\n",
        old_input_text,
        re.IGNORECASE
    )
    if conf_name_match:
        conf_title = conf_name_match.group(1).strip()
        if conf_name_match.group(2): # Acronym was in parentheses
            conf_acronym = conf_name_match.group(2).strip()

    # 2. Extract Website Contents
    # Regex to find each website block
    # Group 1: Number (e.g., "1")
    # Group 2: Acronym part from "Website of ACRONYM_X" (e.g., "CARDIS")
    # Group 3: Number suffix from "_X" (e.g., "0")
    # Group 4: URL
    # Group 5: Content
    website_pattern = re.compile(
        r"(\d+)\.\s+Website of\s+(.*?)_(\d*):\s*(https?://[^\s]+)\s*"
        r"Website information of [^:]*:\s*\n" # Match "Website information of ANYTHING:"
        r"([\s\S]*?)"
        r"(?=(?:\n\s*\d+\.\s+Website of)|\Z)", # Positive lookahead for next block or end of string
        re.IGNORECASE
    )

    websites = []
    first_site_acronym_candidate = None

    for match in website_pattern.finditer(old_input_text):
        # site_number_text = match.group(1) # e.g., "1"
        site_acronym_candidate = match.group(2).strip() # e.g., "CARDIS"
        # site_suffix_num = match.group(3) # e.g., "0"
        url = match.group(4).strip()
        content = match.group(5).strip()
        websites.append({"url": url, "content": content})

        if not first_site_acronym_candidate and site_acronym_candidate:
            first_site_acronym_candidate = site_acronym_candidate

    # If acronym wasn't found with conference name, try to use one from website blocks
    # Only override if conf_acronym is still the default "N/A"
    if conf_acronym == "N/A" and first_site_acronym_candidate:
        conf_acronym = first_site_acronym_candidate

    # 3. Build the new input string
    new_input_parts.append("Conference Info:")
    new_input_parts.append(f"Title: {conf_title}")
    new_input_parts.append(f"Acronym: {conf_acronym}\n") # Extra newline for separation

    new_input_parts.append("Candidate Website Contents:")
    for i, site in enumerate(websites):
        new_input_parts.append(f"Source Link [{i+1}]: {site['url']}")
        new_input_parts.append(f"Content [{i+1}]:\n{site['content']}")
        if i < len(websites) - 1:
            new_input_parts.append("\n---\n")

    return "\n".join(new_input_parts)


def process_csv_file(input_csv_filepath):
    """
    Reads a CSV file, transforms the 'input:' column,
    and parses the 'output:' column.
    Returns a list of dictionaries, where each dictionary represents a row
    with transformed 'input:' and parsed 'output:'.
    """
    processed_rows = []
    
    try:
        with open(input_csv_filepath, 'r', encoding='utf-8', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            if 'input:' not in reader.fieldnames or 'output:' not in reader.fieldnames:
                print(f"Error: CSV file must contain 'input:' and 'output:' columns. Found: {reader.fieldnames}")
                return []

            for row_num, row_dict in enumerate(reader):
                try:
                    old_input = row_dict.get('input:', '') # Use .get for safety
                    output_str = row_dict.get('output:', '{}')

                    new_input = transform_single_input(old_input)
                    
                    try:
                        output_json = json.loads(output_str)
                    except json.JSONDecodeError as e:
                        print(f"Warning: Row {row_num+2} (line number in file) - Could not parse JSON in 'output:' column: {e}")
                        print(f"Original output string: {output_str}")
                        output_json = {"error": "JSON parsing failed", "original_output": output_str}

                    processed_rows.append({
                        'input:': new_input,
                        'output:': output_json
                    })
                except Exception as e:
                    print(f"Error processing row {row_num+2} (line number in file): {e}")
                    # Add a placeholder or skip the row if critical error
                    processed_rows.append({
                        'input:': f"Error processing original input: {row_dict.get('input:', '')}",
                        'output:': {"error": "Row processing failed", "original_input": old_input}
                    })
    except FileNotFoundError:
        print(f"Error: Input CSV file not found at '{input_csv_filepath}'")
        return []
    except Exception as e:
        print(f"An unexpected error occurred while reading the CSV: {e}")
        return []
        
    return processed_rows

def write_transformed_csv(output_csv_filepath, data_to_write):
    """
    Writes the processed data to a new CSV file.
    Each row in data_to_write should be a dictionary with 'input:' and 'output:' keys.
    The 'output:' value (which is a Python dict) will be converted to a JSON string.
    """
    if not data_to_write:
        print("No data to write.")
        return

    try:
        with open(output_csv_filepath, 'w', encoding='utf-8', newline='') as outfile:
            # Ensure 'input:' and 'output:' are the desired column names
            fieldnames = ['input:', 'output:']
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            for p_row in data_to_write:
                # Convert output dict back to JSON string for CSV
                row_to_write = {
                    'input:': p_row.get('input:', ''), # Use .get for safety
                    'output:': json.dumps(p_row.get('output:', {}))
                }
                writer.writerow(row_to_write)
        print(f"\nProcessed data successfully written to '{output_csv_filepath}'")
    except IOError:
        print(f"Error: Could not write to output file '{output_csv_filepath}'. Check permissions or path.")
    except Exception as e:
        print(f"An unexpected error occurred while writing the CSV: {e}")

# --- Main execution ---
if __name__ == "__main__":
    # !!! THAY ĐỔI TÊN FILE CHO PHÙ HỢP !!!
    input_file_path = "D:/NEW-SERVER-TS/src/conference/examples/determine_links_train_data.csv"  # <<<< ---- THAY ĐỔI TÊN FILE NÀY
    output_file_path = "D:/NEW-SERVER-TS/src/conference/examples/transformed_data.csv" # <<<< ---- TÊN FILE CSV ĐẦU RA

    print(f"Processing CSV file: {input_file_path}")
    processed_data = process_csv_file(input_file_path)

    if processed_data:
        # Tùy chọn: In một vài bản ghi đầu tiên để kiểm tra
        print("\n--- Sample of Processed Data ---")
        for i, item in enumerate(processed_data[:2]): # In 2 bản ghi đầu
            print(f"\n--- Record {i+1} ---")
            print("Transformed Input:")
            print(item['input:'])
            print("\nParsed Output:")
            print(json.dumps(item['output:'], indent=2, ensure_ascii=False)) # ensure_ascii=False để hiển thị tiếng Việt
            print("="*40)

        # Ghi kết quả ra file CSV mới
        write_transformed_csv(output_file_path, processed_data)
    else:
        print("No data was processed. Output file will not be created.")

4. Transform extract info

In [62]:
import csv
import re

def transform_single_input_v2(old_input_text):
    """
    Transforms a single 'input:' string from the old format (v2) to the new format (v2).
    Old format example:
    Conference Applied Computing Conference (ACC):
    Skip to content...

    New format example:
    Conference Title: Applied Computing Conference
    Conference Acronym: ACC

    Main Website Content:
    Skip to content...
    """
    new_parts = []
    conference_title = "N/A"
    conference_acronym = "N/A"
    main_content = old_input_text # Default to original if pattern doesn't match

    # Regex to capture title and optional acronym from the first line
    # ^Conference\s+ : Starts with "Conference "
    # (.*?)          : Captures the title (non-greedy) - Group 1
    # (?:\s*\((.*?)\))? : Optional group for acronym:
    #   \s*\(        : Optional space then (
    #   (.*?)        : Captures acronym (non-greedy) - Group 2
    #   \)           : Closing )
    # ?:             : Non-capturing group for the whole acronym part
    # ?              : Makes the acronym part optional
    # :\s*\n?        : Colon, optional space, optional newline (to separate header from content)
    match = re.match(r"^Conference\s+(.*?)(?:\s*\((.*?)\))?:\s*(\n|$)", old_input_text, re.IGNORECASE)

    if match:
        conference_title = match.group(1).strip()
        if match.group(2): # Acronym was found
            conference_acronym = match.group(2).strip()
        
        # The rest of the string after the matched first line is the main content
        # match.end() gives the index after the matched part (including the newline if captured by \n)
        content_start_index = match.end()
        main_content = old_input_text[content_start_index:].strip()
    else:
        # If the first line doesn't match, we might try a simpler split
        # or just use defaults and the whole text as content.
        # For now, let's assume if it doesn't match, the whole thing is content,
        # and title/acronym remain N/A or you can add more sophisticated parsing.
        print(f"Warning: Could not parse conference title/acronym from first line: {old_input_text.splitlines()[0] if old_input_text else 'Empty Input'}")
        # Keep main_content as old_input_text.strip() in this case
        main_content = old_input_text.strip()


    new_parts.append(f"Conference Title: {conference_title}")
    new_parts.append(f"Conference Acronym: {conference_acronym}")
    new_parts.append("") # Blank line
    new_parts.append("Main Website Content:")
    new_parts.append(main_content)

    return "\n".join(new_parts)

def process_and_write_csv_v2(input_filepath, output_filepath):
    """
    Reads a CSV, transforms its 'input:' column using transform_single_input_v2,
    and writes the result to a new CSV, preserving other columns.
    """
    transformed_rows = []
    fieldnames = []

    try:
        with open(input_filepath, 'r', encoding='utf-8', newline='') as infile:
            reader = csv.DictReader(infile)
            fieldnames = reader.fieldnames
            if not fieldnames:
                print(f"Error: CSV file '{input_filepath}' appears to be empty or has no header.")
                return
            if 'input:' not in fieldnames:
                print(f"Error: CSV file '{input_filepath}' must contain an 'input:' column. Found columns: {fieldnames}")
                return

            for i, row in enumerate(reader):
                # Make a copy to modify
                new_row = row.copy()
                old_input_content = row.get('input:', '') # Get content of 'input:' column

                if old_input_content:
                    transformed_content = transform_single_input_v2(old_input_content)
                    new_row['input:'] = transformed_content
                else:
                    # Handle empty input: column if necessary, or leave as is
                    new_row['input:'] = "" 
                    print(f"Warning: Row {i+2} has empty 'input:' field.")

                transformed_rows.append(new_row)
    
    except FileNotFoundError:
        print(f"Error: Input CSV file not found at '{input_filepath}'")
        return
    except Exception as e:
        print(f"An unexpected error occurred while reading the CSV '{input_filepath}': {e}")
        return

    if not transformed_rows:
        print("No data was processed. Output file will not be created.")
        return

    try:
        with open(output_filepath, 'w', encoding='utf-8', newline='') as outfile:
            # Use the fieldnames from the original CSV
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(transformed_rows)
        print(f"Successfully transformed '{input_filepath}' and saved to '{output_filepath}'")
    except IOError:
        print(f"Error: Could not write to output file '{output_filepath}'. Check permissions or path.")
    except Exception as e:
        print(f"An unexpected error occurred while writing the CSV '{output_filepath}': {e}")

# --- Main execution ---
if __name__ == "__main__":
    # !!! THAY ĐỔI TÊN FILE CHO PHÙ HỢP !!!
    input_csv_file = "D:/NEW-SERVER-TS/src/conference/train/extract_infor_done_300_shrink.csv"  # <<<< ---- THAY ĐỔI TÊN FILE CSV ĐẦU VÀO
    output_csv_file = "transformed_second_data.csv" # <<<< ---- TÊN FILE CSV ĐẦU RA MONG MUỐN

    print(f"Starting transformation for {input_csv_file}...")
    process_and_write_csv_v2(input_csv_file, output_csv_file)

    # Để kiểm tra, bạn có thể tạo một file CSV mẫu tên là 'your_second_input_file.csv'
    # với nội dung như sau:
    """
input:,other_column
"Conference Applied Computing Conference (ACC):

Skip to contentsecretariat@computing-conf.org 
Connect With Us:Applied Computing2025MenuEvent | Committees 
Publication Ethics Statement 
Publications 
........","Some other data 1"
"Conference International Symposium on Advanced Security and Privacy (ASAP):
Main content for ASAP
More lines
Even more lines","Another value"
"Conference Another Conference with no Acronym:
Content for no acronym conference.
Still going.","Data3"
"This is an unmatched line:
It will be treated as content.","Error case"
    """
    # Sau khi chạy, 'transformed_second_data.csv' sẽ được tạo.

Starting transformation for D:/NEW-SERVER-TS/src/conference/train/extract_infor_done_300_shrink.csv...
Successfully transformed 'D:/NEW-SERVER-TS/src/conference/train/extract_infor_done_300_shrink.csv' and saved to 'transformed_second_data.csv'
